# Fragment 4

This fragment introduces tenses but breaks negative forms.

This fragment is able to interpret the following sentences from the previous fragment:
* (broken) "John runs and Mary doesn't run" as `(run john)∧¬(run mary)`
* "the dog runs" as `run dog`
* "John loves Mary" as `love john mary`
* "it is necessary that John runs" as `□(run john)`
* (broken) "it is not possible that the dog kills John" as `¬◇(kill dog john)`
* (broken) "it is not the case that John loves Mary" as `¬(love john mary)`
* "John has to run" as `□(run john)`
* (broken) "the dog is not allowed to kill John" as `¬◇(kill dog john)`
* "John has to be allowed to run" as `□◇(run john)`

This fragment is able to interpret the following new sentences:
* "John will run" as `F(run john)`
* "John ran" as `P(run john)`
* "John had run" as `PP(run john)`
* "John will have run" as `FP(run john)`
* "John would run" as `PF(run john)`
* "John would have run" as `PFP(run john)`
* "John had to run" as `P□(run john)`
* "John had had to run" as `PP□(run john)`
* "John will have to run" as `F□(run john)`
* "John will have had to run" as `FP□(run john)`
* "John will be allowed to run" as `F◇(run john)`
* "John was allowed to run" as `P◇(run john)`

## Parsing

In [1]:
abstract TensesGrammar = {
    cat
        S;     -- Sentence
        SMod;  -- Sentence modifier
        Vi;    -- Intransitive verb
        Vt;    -- Transitive verb
        Vmod;  -- Modal verb
        VP;    -- Verb phrase
        Npr;   -- Proper noun
        N;     -- Common noun
        NP;    -- Noun phrase
        Conj;  -- Conjunction
    fun
        -- Positive sentence: "John runs"
        spos : NP -> VP -> S;
        -- Negative sentence: "John doesn't run"
        --sneg : NP -> VP -> S;
        -- Connected sentence:
        -- "John runs and Mary doesn't run"
        sconj : S -> Conj -> S -> S;
        -- Positive modified sentence: "it is necessary that John runs"
        smodpos : SMod -> S -> S;
        -- Negative modified sentence: "it is not necessary that John runs"
        --smodneg : SMod -> S -> S;
        
        -- Build a verb phrase from an intransitive verb: "run"
        vpvi : Vi -> VP;
        -- Build a verb phrase from a transitive verb and a noun phrase
        -- "love" + "Mary" -> "love Mary"
        vpvt : Vt -> NP -> VP;
        -- Apply the past tense operator
        vppast : VP -> VP;
        -- apply the future tense operator
        vpfut : VP -> VP;
        -- Apply a modal verb to an existing verb phrase
        -- "have to" + "run" -> "have to run"
        vpvmod : Vmod -> VP -> VP;
        
        -- Build a noun phrase from a proper noun: "John"
        npnpr : Npr -> NP;
        -- Build a noun phrase from a common noun
        -- "dog" -> "the dog"
        npn : N -> NP;
        
        -- Verbs
        run, jump, laugh, sing, howl, scream : Vi;
        read, poison, eat, like, love, loath, kick, see, understand, kill : Vt;
        haveto, beallowedto : Vmod;
        bepossible, benecessary, bethecase : SMod;
        -- Nouns
        prudence, ethel, chester, jo, bertie, fiona, berta, john, mary, peter : Npr;
        book, cake, katze, golfer, dog, lecturer, student, singer : N;
        -- Conjunctions
        and, or : Conj;
}

Defined TensesGrammar

In [2]:
concrete TensesGrammarEng of TensesGrammar = {
    param
        Tense = Present | Past | Future;
    oper
        -- The type of a verb
        VType : Type = {
            base : Str;
            singleTense : Tense => Str;
            doubleTense : Tense => Tense => Str;
            tripleTense : Tense => Tense => Tense => Str
        };
    lincat
        S = Str;        -- Sentence
        SMod = VType;   -- Sentence modifier
        Vi = VType;     -- Intransitive verb
        Vt = VType;     -- Transitive verb
        Vmod = VType;   -- Modal verb
        VP = VType;     -- Verb phrase
        Npr = Str;      -- Proper noun
        N = Str;        -- Common noun
        NP = Str;       -- Noun phrase
        Conj = Str;     -- Conjunction
    oper
        -- None workaround
        None : Str = "[ERROR]";
        
        -- Helper function to construct a verb
        mkV_ : Str -> Str -> Str -> Str -> VType = \vb,vpart,vpres,vpast -> {
            base = vb;
            singleTense = table {
                Present => vpres;
                Past => vpast;
                Future => "will" ++ vb
            };
            doubleTense = table {
                Present => \\_ => None;
                Past => table {
                    Present => None;
                    Past => "had" ++ vpart;
                    Future => "will have" ++ vpart
                };
                Future => table {
                    Past => "would" ++ vb;
                    _ => None
                }
            };
            tripleTense = table {
                Present => \\_,_ => None;
                Past => table {
                    Present => \\_ => None;
                    Past => \\_ => None;
                    Future => table {
                        Past => "would have" ++ vb;
                        _ => None
                    }
                };
                Future => \\_,_ => None
            }
        };
        
        -- Helper function to construct a verb
        mkV = overload {
            mkV : Str -> Str -> Str -> VType =
                \vb,vpast,vpart -> mkV_ vb vpart (vb + "s") vpast;
            mkV : Str -> Str -> Str -> Str -> VType =
                \vb,vpres,vpast,vpart -> mkV_ vb vpart vpres vpast;
        };
        -- Helper function to construct a be verb
        -- mkBeV "possible" = { pos = "is possible" ; neg = "is not possible"}
        mkBeV : Str -> VType = \s -> mkV ("be" ++ s) ("is" ++ s) ("was" ++ s) ("been" ++ s);
        
        -- Helper function to apply a tense operator to a verb
        applyTense : VType -> Tense -> VType = \v,tense -> {
            base = None;
            singleTense = table {
                Present => v.singleTense ! tense;
                x => v.doubleTense ! tense ! x
            };
            doubleTense = v.tripleTense ! tense;
            tripleTense = \\_,_,_ => None
        };
        
        -- Helper function to append a string to verb
        vpconcat : VType -> Str -> VType = \v,s -> {
            base = v.base ++ s;
            singleTense = \\t => v.singleTense ! t ++ s;
            doubleTense = \\t1,t2 => v.doubleTense ! t1 ! t2 ++ s;
            tripleTense = \\t1,t2,t3 => v.tripleTense ! t1 ! t2 ! t3 ++ s;
        };
    lin
        -- Positive sentence: "John runs"
        -- spos : NP -> VP -> S
        spos npr v = npr ++ v.singleTense ! Present;
        -- Negative sentence: "John doesn't run"
        -- sneg : NP -> VP -> S
        --sneg npr v = npr ++ v.neg;
        -- Connected sentence:
        -- "John runs and Mary doesn't run"
        -- sconj : S -> Conj -> S -> S
        sconj s1 c s2 = s1 ++ c ++ s2;
        -- Positive modified sentence: "it is necessary that John runs"
        -- smodpos : SMod -> S -> S
        smodpos smod s = "it" ++ smod.singleTense ! Present ++ "that" ++ s;
        -- Negative modified sentence: "it is not necessary that John runs"
        -- smodneg : SMod -> S -> S
        --smodneg smod s = "it" ++ smod.neg ++ "that" ++ s;
        
        -- Build a verb phrase from an intransitive verb: "run"
        -- vpvi : Vi -> VP
        vpvi v = v;
        -- Build a verb phrase from a transitive verb and a noun phrase
        -- "love" + "Mary" -> "love Mary"
        -- vpvt : Vt -> NP -> VP
        vpvt v np = vpconcat v np;
        -- Apply the past tense operator
        -- vppast : VP -> VP
        vppast vp = applyTense vp Past;
        -- apply the future tense operator
        -- vpfut : VP -> VP
        vpfut vp = applyTense vp Future;
        -- Apply a modal verb to an existing verb phrase
        -- "have to" + "run" -> "have to run"
        -- vpvmod : Vmod -> VP -> VP
        vpvmod vmod vp = vpconcat vmod vp.base;
        
        -- Build a noun phrase from a proper noun: "John"
        -- npnpr : Npr -> NP
        npnpr npr = npr;
        -- Build a noun phrase from a common noun
        -- "dog" -> "the dog"
        -- npn : N -> NP
        npn n = "the" ++ n;
        
        -- Intrasitive verbs
        run = mkV "run" "ran" "run"; jump = mkV "jump" "jumped" "jumped"; laugh = mkV "laugh" "laughed" "laughed";
        sing = mkV "sing" "sang" "sung"; howl = mkV "howl" "howled" "howled"; scream = mkV "scream" "screamed" "screamed";
        -- Transitive verbs
        read = mkV "read" "read" "read"; poison = mkV "poison" "poisoned" "poisoned"; eat = mkV "eat" "ate" "eaten";
        like = mkV "like" "liked" "liked"; love = mkV "love" "loved" "loved"; loath = mkV "loath" "loathed" "loathed";
        kick = mkV "kick" "kicked" "kicked"; see = mkV "see" "saw" "seen"; understand = mkV "understand" "understood" "understood";
        kill = mkV "kill" "killed" "killed";
        -- Modal verbs
        haveto = mkV "have to" "has to" "had to" "had to"; beallowedto = mkBeV "allowed to";
        bepossible = mkBeV "possible"; benecessary = mkBeV "necessary"; bethecase = mkBeV "the case";
        -- Nouns
        prudence = "Prudence"; ethel = "Ethel"; chester = "Chester"; jo = "Jo"; bertie = "Bertie"; fiona = "Fiona"; berta = "Berta";
        john = "John"; mary = "Mary"; peter = "Peter";
        book = "book"; cake = "cake"; katze = "cat"; golfer = "golfer"; dog = "dog"; lecturer = "lecturer"; student = "student";
        singer = "singer"; 
        -- Conjunctions
        and = "and"; or = "or";
}

### Parsing test cases (old)

In [3]:
parse "John runs"

spos (npnpr john) (vpvi run)

In [4]:
parse "John doesn't run"

The parser failed at token 2: "doesn't"

In [5]:
parse "John runs and Mary doesn't run"

The parser failed at token 5: "doesn't"

In [6]:
parse "John doesn't jump or Mary runs"

The parser failed at token 2: "doesn't"

In [7]:
parse "the dog runs"

spos (npn dog) (vpvi run)

In [8]:
parse "John loves Mary"

spos (npnpr john) (vpvt love (npnpr mary))

In [9]:
parse "John loves Mary and Mary doesn't love John"

The parser failed at token 6: "doesn't"

In [10]:
parse "it is necessary that John runs"

smodpos benecessary (spos (npnpr john) (vpvi run))

In [11]:
parse "it is not possible that the dog kills John"

The parser failed at token 3: "not"

In [12]:
parse "it is not the case that John loves Mary"

The parser failed at token 3: "not"

In [13]:
parse "John has to run"

spos (npnpr john) (vpvmod haveto (vpvi run))

In [14]:
parse "the dog is not allowed to kill John"

The parser failed at token 4: "not"

In [15]:
parse "John has to be allowed to run"

spos (npnpr john) (vpvmod haveto (vpvmod beallowedto (vpvi run)))

### Parsing test cases (new)

In [16]:
parse "John will run"

spos (npnpr john) (vpfut (vpvi run))

In [17]:
parse "John ran"

spos (npnpr john) (vppast (vpvi run))

In [18]:
parse "John had run"

spos (npnpr john) (vppast (vppast (vpvi run)))

In [19]:
parse "John will have run"

spos (npnpr john) (vpfut (vppast (vpvi run)))

In [20]:
parse "John would run"

spos (npnpr john) (vppast (vpfut (vpvi run)))

In [21]:
parse "John would have run"

spos (npnpr john) (vppast (vpfut (vppast (vpvi run))))

In [22]:
parse "John had to run"

spos (npnpr john) (vppast (vpvmod haveto (vpvi run)))

In [23]:
parse "John had had to run"

spos (npnpr john) (vppast (vppast (vpvmod haveto (vpvi run))))

In [24]:
parse "John will have to run"

spos (npnpr john) (vpfut (vpvmod haveto (vpvi run)))

In [25]:
parse "John will have had to run"

spos (npnpr john) (vpfut (vppast (vpvmod haveto (vpvi run))))

In [26]:
parse "John will be allowed to run"

spos (npnpr john) (vpfut (vpvmod beallowedto (vpvi run)))

In [27]:
parse "John was allowed to run"

spos (npnpr john) (vppast (vpvmod beallowedto (vpvi run)))

## Semantics construction

In [28]:
theory plnq : ur:?LF =
    proposition : type ❘ # o ❙
    individual : type ❘ # ι ❙
    
    not : o ⟶ o         ❘ # ¬ 1 prec 100 ❙
    and : o ⟶ o ⟶ o     ❘ # 1 ∧ 2 prec 90 ❙
    
    or : o ⟶ o ⟶ o      ❘ # 1 ∨ 2 prec 80
                        ❘ = [a,b] ¬ (¬ a ∧ ¬ b) ❙
❚

theory modalplnq : ur:?LF =
    include ?plnq ❙
    box : o ⟶ o     ❘ # □ 1 prec 100 ❙
    diamond : o ⟶ o ❘ # ◇ 1 prec 100 ❙
❚

theory tenseplnq : ur:?LF =
    include ?modalplnq ❙
    
    // FΦ: it will at some stage in the future be the case that Φ ❙
    futF : o ⟶ o ❘ # F 1 prec 100 ❙
    // PΦ: it was at some stage in the past the case that Φ ❙
    pastP : o ⟶ o ❘ # P 1 prec 100 ❙
❚

theory plnqTest : ?tenseplnq =
    // individuals ❙
    prudence : ι ❙ ethel : ι ❙ chester : ι ❙ jo : ι ❙ bertie : ι ❙ fiona : ι ❙ berta : ι ❙ john : ι ❙ mary : ι ❙ peter : ι ❙
    book : ι ❙ cake : ι ❙ cat : ι ❙ golfer : ι ❙ dog : ι ❙ lecturer : ι ❙ student : ι ❙ singer : ι ❙
    // verbs ❙
    run : ι ⟶ o ❙ jump : ι ⟶ o ❙ laugh : ι ⟶ o ❙ sing : ι ⟶ o ❙ howl : ι ⟶ o ❙ scream : ι ⟶ o ❙
    read : ι ⟶ ι ⟶ o ❙ poison : ι ⟶ ι ⟶ o ❙ eat : ι ⟶ ι ⟶ o ❙ like : ι ⟶ ι ⟶ o ❙ love : ι ⟶ ι ⟶ o ❙ loath : ι ⟶ ι ⟶ o ❙
    kick : ι ⟶ ι ⟶ o ❙ see : ι ⟶ ι ⟶ o ❙ understand : ι ⟶ ι ⟶ o ❙ kill : ι ⟶ ι ⟶ o ❙
❚

Created theory plnq

In [29]:
view semantics : http://mathhub.info/comma/jupyter/TensesGrammar.gf?TensesGrammar -> ?plnqTest =
    S = o ❙             // Sentence ❙
    SMod = o ⟶ o ❙      // Sentence modifier ❙
    Vi = ι ⟶ o ❙        // Intransitive verb ❙
    Vt = ι ⟶ ι ⟶ o ❙    // Transitive verb ❙
    Vmod = o ⟶ o ❙      // Modal verb ❙
    VP = ι ⟶ o ❙        // Verb phrase ❙
    Npr = ι ❙           // Proper noun ❙
    N = ι ❙             // Common noun, as a simplification modeled incorrectly
                        // as an individual ❙
    NP = ι ❙            // Noun phrase ❙
    Conj = o ⟶ o ⟶ o ❙  // Conjunction ❙
    
    // Positive sentence: "John runs"
    // spos : Npr -> V -> S ❙
    spos = [npr, v] v npr ❙
    // Negative sentence: "John doesn't run"
    // sneg : Npr -> V -> S ❙
    //sneg = [npr, v] ¬(v npr) ❙
    // Connected sentence:
    // "John runs and Mary doesn't run"
    // sconj : S -> Conj -> S -> S ❙
    sconj = [s1, conj, s2] conj s1 s2 ❙
    // Positive modified sentence: "it is necessary that John runs"
    // smodpos : SMod -> S -> S ❙
    smodpos = [smod, s] smod s ❙
    // Negative modified sentence: "it is not necessary that John runs"
    // smodneg : SMod -> S -> S ❙
    //smodneg = [smod, s] ¬(smod s) ❙
    
    // Build a verb phrase from an intransitive verb: "run"
    // vpvi : Vi -> VP ❙
    vpvi = [v] v ❙
    // Build a verb phrase from a transitive verb and a noun phrase
    // "love" + "Mary" -> "love Mary"
    // vpvt : Vt -> NP -> VP ❙
    vpvt = [vt, addressee] [actor] vt actor addressee ❙
    // Apply the past tense operator
    // vppast : VP -> VP ❙
    vppast = [v] [i] P (v i) ❙
    // apply the future tense operator
    // vpfut : VP -> VP ❙
    vpfut = [v] [i] F (v i) ❙
    // Apply a modal verb to an existing verb phrase
    // "have to" + "run" -> "have to run"
    // vpvmod : Vmod -> VP -> VP ❙
    vpvmod = [vmod, vp] [i] vmod (vp i) ❙
        
    // Build a noun phrase from a proper noun: "John"
    // npnpr : Npr -> NP ❙
    npnpr = [npr] npr ❙
    // Build a noun phrase from a common noun
    // "dog" -> "the dog"
    // npn : N -> NP ❙
    npn = [n] n ❙
    
    // Verbs ❙
    run = run ❙ jump = jump ❙ laugh = laugh ❙ sing = sing ❙ howl = howl ❙ scream = scream ❙
    read = read ❙ poison = poison ❙ eat = eat ❙ like = like ❙ love = love ❙ loath = loath ❙ kick = kick ❙ see = see ❙ understand = understand ❙ kill = kill ❙
    haveto = box ❙ beallowedto = diamond ❙
    bepossible = diamond ❙ benecessary = box ❙ bethecase = [p] p ❙
    // Nouns ❙
    prudence = prudence ❙ ethel = ethel ❙ chester = chester ❙ jo = jo ❙ bertie = bertie ❙ fiona = fiona ❙ berta = berta ❙ john = john ❙ mary = mary ❙ peter = peter ❙
    book = book ❙ cake = cake ❙ katze = cat ❙ golfer = golfer ❙ dog = dog ❙ lecturer = lecturer ❙ student = student ❙ singer = singer ❙
    // Conjunctions ❙
    and = and ❙ or = or ❙
❚

Created view semantics

### Semantics construction test cases (old)

In [30]:
parse "John runs" | construct

run john

In [31]:
parse "the dog runs" | construct

run dog

In [32]:
parse "John loves Mary" | construct

love john mary

In [33]:
parse "it is necessary that John runs" | construct

□(run john)

In [34]:
parse "John has to run" | construct

□(run john)

In [35]:
parse "John has to be allowed to run" | construct

□◇(run john)

### Semantics construction test cases (new)

In [36]:
parse "John will run" | construct

F (run john)

In [37]:
parse "John ran" | construct

P (run john)

In [38]:
parse "John had run" | construct

P P (run john)

In [39]:
parse "John will have run" | construct

F P (run john)

In [40]:
parse "John would run" | construct

P F (run john)

In [41]:
parse "John would have run" | construct

P F P (run john)

In [42]:
parse "John had to run" | construct

P □(run john)

In [43]:
parse "John had had to run" | construct

P P □(run john)

In [44]:
parse "John will have to run" | construct

F □(run john)

In [45]:
parse "John will have had to run" | construct

F P □(run john)

In [46]:
parse "John will be allowed to run" | construct

F ◇(run john)

In [47]:
parse "John was allowed to run" | construct

P ◇(run john)